# Fermi-Hubbard Hamiltonian
___
This section creates the Fermi-Hubbard Hamiltonian with OpenFermion

In [1]:
# from scipy import sparse
from scipy.sparse import linalg
from openfermion import fermi_hubbard, jordan_wigner, get_sparse_operator

In [2]:
"""Define the Hamiltonian."""
# Parameters from the paper

nsites = 4
U = 10.0

# Hopping Integral

# In the paper U/t = 4
t = U / 4

# Is the sign of the tunneling right?
fermionic_hamiltonian = fermi_hubbard(1, nsites, tunneling=-t, coulomb=U, periodic=False)

# Map to QubitOperator using the JW transform
jw_hamiltonian = jordan_wigner(fermionic_hamiltonian)

# Convert to Scipy sparse matrix
hamiltonian_jw_sparse = get_sparse_operator(jw_hamiltonian)

# Compute ground energy
eigs, _ = linalg.eigsh(hamiltonian_jw_sparse, k=1, which="SA")
ground_energy = eigs[0]

print(f"Ground State Energy: {ground_energy}")
print("JWT transformed Hamiltonian:")
print(jw_hamiltonian)


Ground State Energy: -6.562355678777163
JWT transformed Hamiltonian:
(10+0j) [] +
(1.25+0j) [X0 Z1 X2] +
(1.25+0j) [Y0 Z1 Y2] +
(-2.5+0j) [Z0] +
(2.5+0j) [Z0 Z1] +
(1.25+0j) [X1 Z2 X3] +
(1.25+0j) [Y1 Z2 Y3] +
(-2.5+0j) [Z1] +
(1.25+0j) [X2 Z3 X4] +
(1.25+0j) [Y2 Z3 Y4] +
(-2.5+0j) [Z2] +
(2.5+0j) [Z2 Z3] +
(1.25+0j) [X3 Z4 X5] +
(1.25+0j) [Y3 Z4 Y5] +
(-2.5+0j) [Z3] +
(1.25+0j) [X4 Z5 X6] +
(1.25+0j) [Y4 Z5 Y6] +
(-2.5+0j) [Z4] +
(2.5+0j) [Z4 Z5] +
(1.25+0j) [X5 Z6 X7] +
(1.25+0j) [Y5 Z6 Y7] +
(-2.5+0j) [Z5] +
(-2.5+0j) [Z6] +
(2.5+0j) [Z6 Z7] +
(-2.5+0j) [Z7]


# Qulacs Simulation

In [7]:
"""Convert OpenFermion Hamiltonian to Qulacs Observable"""

from qulacs.observable import create_observable_from_openfermion_text
# from openfermion.utils import count_qubits

qulacs_hamiltonian = create_observable_from_openfermion_text(str(jw_hamiltonian))

n_qubits = qulacs_hamiltonian.get_qubit_count()


print(f"Succesfully created Qulacs Hamiltonian from OpenFermion Hamiltonian on {n_qubits} qubits")



Succesfully created Qulacs Hamiltonian from OpenFermion Hamiltonian on 8 qubits


In [83]:
from qulacs import QuantumState

state = QuantumState(n_qubits)
# state.set_Haar_random_state()

# setting up HF state
state.set_computational_basis(0b11000000)

qulacs_hamiltonian.get_expectation_value(state)

10.0

In [97]:
from qulacs import QuantumCircuit

circuit = QuantumCircuit(n_qubits+1)

# What is the angle?
angle = 0.1

# time steps
t_slice = 1

circuit.add_observable_rotation_gate(qulacs_hamiltonian, angle, t_slice)

print(circuit)

*** Quantum Circuit Info ***
# of qubit: 9
# of step : 21
# of gate : 25
# of 1 qubit gate: 8
# of 2 qubit gate: 4
# of 3 qubit gate: 12
Clifford  : no
Gaussian  : no


